# **VERİ ANALİZİ**

In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import plotly.graph_objs as go
import plotly.express as px

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 500)

In [4]:
import pandas as pd

# Dosyanın tam yolunu belirtin
df = pd.read_csv("C:\\Users\\baryo\\OneDrive\\Masaüstü\\sirketiciyipranma.csv") 

# İlk birkaç satırı görüntüleyin
print(df.head())


   isten_ayrilma       is_seyahati             departman cinsiyet                 is_rolu medeni_durum fazla_mesai  yas  aylik_ucret  evden_uzaklik  son_terfiden_bu_yana_gecen_yil  mevcut_rolde_gecirilen_yil
0              1   Seyahat_Nadiren                 Satis    Kadin        Satis Yoneticisi        Bekar        Evet   41        19479              1                               0                           4
1              0  Seyahat_Siklikla  Arastirma_Gelistirme    Erkek  Arastirma Bilim Insani         Evli       Hayir   49        24907              8                               1                           7
2              1   Seyahat_Nadiren  Arastirma_Gelistirme    Erkek  Laboratuvar Teknisyeni        Bekar        Evet   37         2396              2                               0                           0
3              0  Seyahat_Siklikla  Arastirma_Gelistirme    Kadin  Arastirma Bilim Insani         Evli        Evet   33        23159              3                     

In [5]:
# Keşifçi Veri Analizi (EDA)
def check_df(dataframe, head=5):
    print("##################### Shape #####################")
    print(dataframe.shape)

    print("##################### Types #####################")
    print(dataframe.dtypes)

    print("##################### Head #####################")
    print(dataframe.head(head))

    print("##################### Tail #####################")
    print(dataframe.tail(head))

    print("##################### NA #####################")
    print(dataframe.isnull().sum())

    print("##################### Describe #####################")
    print(dataframe.describe().T)

# Kategorik ve Sayısal Değişkenlerin Belirlenmesi

def grab_col_names(dataframe, cat_th=10, car_th=20):
    cat_cols = [col for col in dataframe.columns if dataframe[col].dtypes == "O"]

    num_but_cat = [col for col in dataframe.columns if dataframe[col].nunique() < cat_th and
                   dataframe[col].dtypes != "O"]

    cat_but_car = [col for col in dataframe.columns if dataframe[col].nunique() > car_th and
                   dataframe[col].dtypes == "O"]

    cat_cols = cat_cols + num_but_cat

    cat_cols = [col for col in cat_cols if col not in cat_but_car]

    num_cols = [col for col in dataframe.columns if dataframe[col].dtypes != "O"]

    num_cols = [col for col in num_cols if col not in num_but_cat]

    print(f"Observations: {dataframe.shape[0]}")
    print(f"Variables: {dataframe.shape[1]}")
    print(f'cat_cols: {len(cat_cols)}')
    print(f'num_cols: {len(num_cols)}')
    print(f'cat_but_car: {len(cat_but_car)}')
    print(f'num_but_cat: {len(num_but_cat)}')

    return cat_cols, num_cols, cat_but_car

# Kategorik Değişken Analizi

def cat_summary_plotly(dataframe, col_name):
    summary_df = dataframe[col_name].value_counts().reset_index()
    summary_df.columns = ['value', 'count']
    summary_df['percentage'] = 100 * summary_df['count'] / len(dataframe)
    summary_df['percentage'] = summary_df['percentage'].round(2)  # 2 decimal places

    fig = px.bar(summary_df, x='value', y='count', text='percentage',
                 title=f'{col_name} Count and Percentage',
                 labels={'value': col_name, 'count': 'Count', 'percentage': 'Percentage'})
    fig.show()

# Sayısal Değişken Analizi

def num_summary_plotly(dataframe, numerical_col):
    fig = px.histogram(dataframe, x=numerical_col, nbins=30,
                       title=f'{numerical_col} Distribution')
    fig.show()

    fig = px.box(dataframe, y=numerical_col, title=f'{numerical_col} Box Plot')
    fig.show()

# Hedef Değişken Analizi (Kategorik)

def target_summary_with_cat_plotly(dataframe, target, categorical_col):
    summary_df = dataframe.groupby(categorical_col)[target].mean().reset_index()

    fig = px.bar(summary_df, x=categorical_col, y=target,
                 title=f'{categorical_col} vs. {target}',
                 labels={categorical_col: categorical_col, target: target})
    fig.show()

# Hedef Değişken Analizi (Sayısal)

def target_summary_with_num_plotly(dataframe, target, numerical_col):
    summary_df = dataframe.groupby(target)[numerical_col].mean().reset_index()

    fig = px.bar(summary_df, x=target, y=numerical_col,
                 title=f'{target} vs. Mean of {numerical_col}',
                 labels={target: target, numerical_col: 'Mean of ' + numerical_col})
    fig.show()

def correlation_heatmap(dataframe):
    corr_matrix = dataframe.corr()

    fig = go.Figure(data=go.Heatmap(
        z=corr_matrix.values,
        x=corr_matrix.index.values,
        y=corr_matrix.columns.values,
        colorscale='Viridis',
        colorbar=dict(title='Correlation'),
    ))

    fig.update_layout(
        title='Correlation Heatmap',
        xaxis=dict(title='Features'),
        yaxis=dict(title='Features')
    )

    fig.show()

# Eksik veya Açıklanmayan Verilerin İncelenmesi
def check_missing_data_plotly(dataframe):
    missing_data = dataframe.isnull().sum().sort_values(ascending=False)
    missing_data = missing_data[missing_data > 0]
    if missing_data.empty:
        print("Veri setinde eksik veya açıklanmayan veri yok.")
    else:
        print("Eksik veya açıklanmayan verilerin sayısı ve oranları:\n", missing_data)
        # Eksik verilerin oranlarını da görselleştirebiliriz
        fig = px.bar(x=missing_data.index, y=missing_data.values, labels={'x': 'Değişkenler', 'y': 'Eksik Veri Sayısı'},
                     title='Eksik Veri Sayısı')
        fig.show()
        
# Aykırı Değer Analizi ve IQR yöntemi ile tespit
def outlier_analysis_iqr_plotly(dataframe, numerical_cols):
    outlier_dict = {}

    for col in numerical_cols:
        Q1 = dataframe[col].quantile(0.25)
        Q3 = dataframe[col].quantile(0.75)
        IQR = Q3 - Q1

        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        outliers = dataframe[(dataframe[col] < lower_bound) | (dataframe[col] > upper_bound)]
        outlier_dict[col] = outliers

        # Boxplot oluşturma
        fig = px.box(dataframe, y=col, title=f'{col} Aykırı Değer Analizi (IQR Method)')
        fig.add_shape(type="line", x0=lower_bound, y0=lower_bound, x1=lower_bound, y1=upper_bound, line=dict(color="red", width=2))
        fig.add_shape(type="line", x0=upper_bound, y0=lower_bound, x1=upper_bound, y1=upper_bound, line=dict(color="red", width=2))
        fig.show()

    return outlier_dict

In [6]:
check_df(df)

##################### Shape #####################
(1470, 12)
##################### Types #####################
isten_ayrilma                      int64
is_seyahati                       object
departman                         object
cinsiyet                          object
is_rolu                           object
medeni_durum                      object
fazla_mesai                       object
yas                                int64
aylik_ucret                        int64
evden_uzaklik                      int64
son_terfiden_bu_yana_gecen_yil     int64
mevcut_rolde_gecirilen_yil         int64
dtype: object
##################### Head #####################
   isten_ayrilma       is_seyahati             departman cinsiyet                 is_rolu medeni_durum fazla_mesai  yas  aylik_ucret  evden_uzaklik  son_terfiden_bu_yana_gecen_yil  mevcut_rolde_gecirilen_yil
0              1   Seyahat_Nadiren                 Satis    Kadin        Satis Yoneticisi        Bekar        Evet   41       

In [17]:
print(f"Kategorik Değişkenler: {cat_cols}")
print(f"Sayısal Değişkenler: {num_cols}")     
# print(f"Kategorik Ancak Kardinal Değişkenler: {cat_but_car}")

Kategorik Değişkenler: ['is_seyahati', 'departman', 'cinsiyet', 'is_rolu', 'medeni_durum', 'fazla_mesai']
Sayısal Değişkenler: ['yas', 'aylik_ucret', 'evden_uzaklik', 'son_terfiden_bu_yana_gecen_yil', 'mevcut_rolde_gecirilen_yil']


# **Kategorik ve sayısal değişkenlerin belirlenmesi**

In [8]:
cat_cols, num_cols, cat_but_car = grab_col_names(df)

cat_cols.remove("isten_ayrilma") #burada neden incelemiyoruz ve kolonu siliyoruz target olduğu için mi ? s

Observations: 1470
Variables: 12
cat_cols: 7
num_cols: 5
cat_but_car: 0
num_but_cat: 1


# **Kategorik Değişken Analizi**


In [9]:
for col in cat_cols:
    cat_summary_plotly(df, col)

# **Sayısal Değişken Analizi**


In [10]:
for col in num_cols:
    num_summary_plotly(df, col)

# **Hedef Değişken Analizi (Kategorik)**

In [11]:
for col in cat_cols:
    target_summary_with_cat_plotly(df,"isten_ayrilma", col)

# **Hedef Değişken Analizi (Sayısal)**

In [12]:
for col in num_cols:
    target_summary_with_num_plotly(df, "isten_ayrilma", col)

# **Korealasyon Analizi**

In [13]:
numerical_df = df[num_cols]
correlation_heatmap(numerical_df)

# **Eksik Verilerin İncelenmesi**

In [14]:
check_missing_data_plotly(df)

Veri setinde eksik veya açıklanmayan veri yok.


# **Aykırı Değer Analizi**

In [15]:
# Aykırı değerleri bul, ayır ve grafiği göster
outliers_dict = outlier_analysis_iqr_plotly(df, num_cols)

In [16]:
# Her bir sayısal değişken için aykırı değerleri içeren DataFrame'i görüntüle
for col, outliers_df in outliers_dict.items():
    print(f"{col} Aykırı Değerler: {len(outliers_df)}")
    print(outliers_df.head())
    print("\n")

yas Aykırı Değerler: 0
Empty DataFrame
Columns: [isten_ayrilma, is_seyahati, departman, cinsiyet, is_rolu, medeni_durum, fazla_mesai, yas, aylik_ucret, evden_uzaklik, son_terfiden_bu_yana_gecen_yil, mevcut_rolde_gecirilen_yil]
Index: []


aylik_ucret Aykırı Değerler: 0
Empty DataFrame
Columns: [isten_ayrilma, is_seyahati, departman, cinsiyet, is_rolu, medeni_durum, fazla_mesai, yas, aylik_ucret, evden_uzaklik, son_terfiden_bu_yana_gecen_yil, mevcut_rolde_gecirilen_yil]
Index: []


evden_uzaklik Aykırı Değerler: 0
Empty DataFrame
Columns: [isten_ayrilma, is_seyahati, departman, cinsiyet, is_rolu, medeni_durum, fazla_mesai, yas, aylik_ucret, evden_uzaklik, son_terfiden_bu_yana_gecen_yil, mevcut_rolde_gecirilen_yil]
Index: []


son_terfiden_bu_yana_gecen_yil Aykırı Değerler: 107
    isten_ayrilma       is_seyahati             departman cinsiyet                 is_rolu medeni_durum fazla_mesai  yas  aylik_ucret  evden_uzaklik  son_terfiden_bu_yana_gecen_yil  mevcut_rolde_gecirilen_yil
15  